In [1]:
import os
import torch
import numpy as np
import pickle
import time
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
from model_manager import CTRLmodel
from tokenizer import Tokenizer
from generation_manager import GeneratorManager

In [2]:
GPU = torch.cuda.is_available()
# if you don't want to use GPU (but you have it aviable) modify in pytorch_transformer.py module the related variable
print(GPU)

True


In [3]:
tokenizer = Tokenizer()
print('tokenizer loaded')

curr_model_path = 'ckpt/training_ckpt_4/model_only_state_dict_v0Last_lr0001.pth' # model checkpoint
print('loading model from:', curr_model_path)

model = CTRLmodel()
reader = model.loadCheckpoint(model_path=curr_model_path)

# Saving only state_dict:
#torch.save(model.state_dict(), model_state_dict_path)
#model_state_dict_path = load_model_path + 'model_only_state_dict_v0Last.pth'
#print('Model state dict saved')
            
if GPU:
    model = model.cuda()
    print('previous checkpoint loaded in GPU')
else: 
    print('previous checkpoint loaded in CPU')

optimizer = torch.optim.Adam(model.parameters()) #lr, betas
model.eval()
print('model ready for inference')

tokenizer loaded
loading model from: ckpt/training_ckpt_4/model_only_state_dict_v0Last_lr0001.pth
MODEL SIZE: 
1280
Found PyTorch checkpoint at  ckpt/training_ckpt_4/model_only_state_dict_v0Last_lr0001.pth
Loading instead of converting from TensorFlow
previous checkpoint loaded in GPU
model ready for inference


In [4]:
def save_sequences(predicted, predicted_stopped):
    data_dir = "Generation_PF00959_model_lr_00001_LAST"
    query = 'After_0.50_percent_penalty_0_key_only_lyso'
    os.makedirs(data_dir, exist_ok=True)
    
    predicted_data_file = os.path.join(data_dir, "predicted_data_" + query + ".p")
    with open(predicted_data_file, "wb") as file:
        pickle.dump(predicted, file)

    predicted_stopped_data_file = os.path.join(data_dir, "predicted_stopped_data_" + query + ".p")
    with open(predicted_stopped_data_file, "wb") as file:
        pickle.dump(predicted_stopped, file)
        

In [5]:
# GENERATION parameters
temperature = 0.9
penalty = 0
top_p = 0.5
np.random.seed(1337)
torch.manual_seed(1337)

def predict_fn(inputs):
    with torch.no_grad():
        inputs = torch.tensor(inputs)
        if GPU:
            inputs = inputs.cuda()            
        output = model(inputs)
        stop_token = output[:, :, 1] # the stop token logits
        output = output[:,:,-26:-1] # remove non-AA token logits
        return output, stop_token
        
generator = GeneratorManager(predict_fn, penalty=penalty, top_p=top_p, temperature=temperature)

In [6]:
predicted = []
predicted_stopped = []
# true_tokens_index_in_probs_all = []
# tokens_probs_all = []
# true_value = []
seq_number = 0
#for entry in random_selection:
while seq_number <= 1500:
    # input_seq = entry['sequence']['metadata']['sequence']
    # FOR TESTIG the code:
    # input_seq = input_seq[0:24]
    input_seq = ''
    
    # if no tax keys:
    tax_lineage = [0]

    offset = 0
    print('generating sequences...')
    start_time = time.time()
    # res, tokens_prob = generation_complete_sequence(input_seq, offset, tax_lineage)
    res, tokens_generated_stopped = generator.generation_complete_sequence(input_seq, offset, tax_lineage)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Done. Time taken: {elapsed_time} seconds.")

    # print("Input: ", input_seq)
    # print("Res: ", res)
    # true_value.append(input_seq)
    # true_tokens_index_in_prob = []
    # true_tokens_index_in_probs_all.append(true_tokens_index_in_prob)
    # tokens_probs_all.append(tokens_prob)
    # print('input_offset should be nothing: ', input_seq[:offset])
    predicted.append(input_seq[:offset] + res)
    if tokens_generated_stopped:
        for tmp_seq in tokens_generated_stopped:
            predicted_stopped.append(input_seq[:offset] + tmp_seq)

    seq_number += 1
    print('hello, this is seq_number', seq_number)
    print('hello, this is seq_number module: ', seq_number%10)
    
    # FOR TESTIG:
    break
    
    if (seq_number%10) == 0:
        print('the model has generated: ', seq_number , ' sequences')
        print('saving sequences')
        save_sequences(predicted, predicted_stopped)
        
print('GENERATION ENDED')
    

generating sequences...
Done. Time taken: 45.20010232925415 seconds.
hello, this is seq_number 1
hello, this is seq_number module:  1
GENERATION ENDED
